In [215]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import time
from collections import defaultdict

In [216]:
x = pd.read_csv("order_brush_order.csv")

In [217]:
x.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [218]:
x["event_time"] = pd.to_datetime(x.event_time)

In [219]:
x = x.sort_values(by='event_time')

In [220]:
x.head()

,orderid,shopid,userid,event_time
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00
146598,31075201870570,190969466,170182475,2019-12-27 00:00:02
80355,31075205798264,2859407,12532131,2019-12-27 00:00:05


In [224]:
c = defaultdict(set)
d = {}
for i, row in x[0:222750].iterrows(): #row[1] == shopid
    shopid = row[1]
    userid = row[2]
    time = row[3]
    if shopid not in d:
        d[shopid] = [(time,userid)] #0:time 1:userid
    else:
        d[shopid] += [(time,userid)]
        count = defaultdict(int)
        for j in reversed(range(len(d[shopid]))):
            if timedelta.total_seconds(time - d[shopid][j][0]) > 3600: # check if not within an hour
                break
            else:
                count[d[shopid][j][1]] += 1 # count(userid) + 1
 
        for k in count: # read counts
            if count[k] >= 3:
                c[shopid].add(k) # shopid.append(userid)
                

In [225]:
s = pd.DataFrame({"shopid": x["shopid"].unique(), "userid": "0"})
for i in range(18770):
    a = s.iloc[i,0]
    if a in c:
        s.iloc[i,1] = "&".join(str(i) for i in c[a])

In [227]:
s.to_csv("export.csv")